In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
#from yellowbrick.features import PCA
import gc
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
dns1 = pd.read_csv("01-12/DrDoS_DNS.csv")
ldap1 = pd.read_csv('01-12/DrDoS_LDAP.csv')
mssql1 = pd.read_csv('01-12/DrDoS_MSSQL.csv')
netbois1 = pd.read_csv('01-12/DrDoS_NetBIOS.csv')
ntp = pd.read_csv('01-12/DrDoS_NTP.csv')
snmp = pd.read_csv('01-12/DrDoS_SNMP.csv')
ssdp = pd.read_csv('01-12/DrDoS_SSDP.csv')
udp1 = pd.read_csv('01-12/DrDoS_UDP.csv')
syn1 = pd.read_csv('01-12/Syn.csv')
tftp = pd.read_csv('01-12/TFTP.csv')
udplag1 = pd.read_csv('01-12/UDPLag.csv') 
ldap2 = pd.read_csv('03-11/LDAP.csv')
mssql2 = pd.read_csv('03-11/MSSQL.csv')
netbios = pd.read_csv('03-11/NetBIOS.csv')
portmap = pd.read_csv('03-11/Portmap.csv')
syn2 = pd.read_csv('03-11/Syn.csv')
udp2 = pd.read_csv('03-11/UDP.csv')
udplag2 = pd.read_csv('03-11/UDPLag.csv') 

In [3]:
data = pd.concat([dns1, ldap1, mssql1, netbois1, ntp, snmp, ssdp, udp1, syn1, tftp, udplag1,
                 ldap2, mssql2, netbios, portmap, syn2, udp2, udplag2])

In [4]:
del dns1
del ldap1
del mssql1
del netbois1
del ntp
del snmp
del ssdp
del udp1
del syn1
del tftp
del udplag1
del ldap2
del mssql2
del netbios
del portmap
del syn2
del udp2
del udplag2
gc.collect()

20

In [5]:
data[' Label'].value_counts()

TFTP             20082580
Syn               6473789
MSSQL             5787453
DrDoS_SNMP        5159870
DrDoS_DNS         5071011
DrDoS_MSSQL       4522492
DrDoS_NetBIOS     4093279
UDP               3867155
NetBIOS           3657497
DrDoS_UDP         3134645
DrDoS_SSDP        2610611
DrDoS_LDAP        2179930
LDAP              1915122
DrDoS_NTP         1202642
UDP-lag            366461
Portmap            186960
BENIGN             113828
UDPLag               1873
WebDDoS               439
Name:  Label, dtype: int64

In [6]:
data[' Label'] = data[' Label'].replace({'DrDoS_DNS':'DNS', 'DrDoS_LDAP':'LDAP', 'DrDoS_MSSQL':'MSSQL',
                                        'DrDoS_NetBIOS':'NetBIOS', 'DrDoS_NTP':'NTP', 'DrDoS_SNMP':'SNMP',
                                        'DrDoS_SSDP':'SSDP', 'DrDoS_UDP':'UDP', 'UDP-lag':'UDPLag'})

In [7]:
data[' Label'].value_counts()

TFTP       20082580
MSSQL      10309945
NetBIOS     7750776
UDP         7001800
Syn         6473789
SNMP        5159870
DNS         5071011
LDAP        4095052
SSDP        2610611
NTP         1202642
UDPLag       368334
Portmap      186960
BENIGN       113828
WebDDoS         439
Name:  Label, dtype: int64

In [8]:
#data.count
#[70427637 rows x 88 columns]

In [9]:
# data sample
col = list(data.columns)
for i in range(len(data.iloc[0])):
    print(col[i], '\t\t\t', data.iloc[0][i])

Unnamed: 0 			 425
Flow ID 			 172.16.0.5-192.168.50.1-634-60495-17
 Source IP 			 172.16.0.5
 Source Port 			 634
 Destination IP 			 192.168.50.1
 Destination Port 			 60495
 Protocol 			 17
 Timestamp 			 2018-12-01 10:51:39.813448
 Flow Duration 			 28415
 Total Fwd Packets 			 97
 Total Backward Packets 			 0
Total Length of Fwd Packets 			 42680.0
 Total Length of Bwd Packets 			 0.0
 Fwd Packet Length Max 			 440.0
 Fwd Packet Length Min 			 440.0
 Fwd Packet Length Mean 			 440.0
 Fwd Packet Length Std 			 0.0
Bwd Packet Length Max 			 0.0
 Bwd Packet Length Min 			 0.0
 Bwd Packet Length Mean 			 0.0
 Bwd Packet Length Std 			 0.0
Flow Bytes/s 			 1502023.5790955485
 Flow Packets/s 			 3413.689952489882
 Flow IAT Mean 			 295.98958333333337
 Flow IAT Std 			 500.95930068517794
 Flow IAT Max 			 3596.0
 Flow IAT Min 			 1.0
Fwd IAT Total 			 28415.0
 Fwd IAT Mean 			 295.98958333333337
 Fwd IAT Std 			 500.95930068517794
 Fwd IAT Max 			 3596.0
 Fwd IAT Min 			 1.0
Bwd IAT Tota

### Handling outliers (won't work)

cols = list(data.columns)
cols.remove('Unnamed: 0')
cols.remove('Flow ID')
cols.remove(' Source IP')
cols.remove(' Destination IP')
cols.remove(' Timestamp')
cols.remove(' Label')
for c in cols:
    q1 = np.percentile(data[c],25)
    q3 = np.percentile(data[c],75)
    out_limit = 1.5 * (q3 - q1)
    out_cols = data[(data[c] < q1 - out_limit) | (data[c] > q3 + out_limit)].index
    print(c, out_cols)


Unnamed: 0

### Feature Engineering

In [10]:
source_ip = [int(''.join([bin(int(x)+256)[3:] for x in ip.split('.')]),2) for ip in data[' Source IP']]
data[' Source IP'] = pd.Series(source_ip)

In [11]:
dest_ip = [int(''.join([bin(int(x)+256)[3:] for x in ip.split('.')]),2) for ip in data[' Destination IP']]
data[' Destination IP'] = pd.Series(dest_ip)

In [12]:
data_Y = data[' Label'].to_numpy().flatten()
data_X = data.drop(columns=['Flow ID',' Timestamp', ' Label'])
del data
gc.collect()

60

### PCA

In [ ]:
pca = PCA(n_components=2)
data_X_pca = pca.fit_transform(data_X)

In [ ]:
plt.scatter(data_X_pca[:, 0], data_X_pca[:, 1],marker='o')
plt.show()

visualizer2d = PCA(scale=False, classes=list(np.unique(data_Y)))
visualizer2d.fit_transform(data_X, data_Y)
visualizer2d.show()

### Some EDA

In [ ]:
data['Unnamed: 0'].value_counts()

In [ ]:
data['Unnamed: 0'].value_counts().sort_index().plot(style='.')

Source IP

In [ ]:
data[' Source IP'].value_counts()

Source Port

In [ ]:
data[' Source Port'].value_counts()

In [ ]:
data[' Source Port'].value_counts().sort_index().plot(style='.')

Destination Port

In [ ]:
data[' Destination Port'].value_counts()

In [ ]:
data[' Destination Port'].value_counts().sort_index().plot(style='.')

Protocol

In [ ]:
data[' Protocol'].value_counts()

Flow Duration

In [ ]:
data[' Flow Duration'].value_counts()

In [ ]:
data[' Flow Duration'].value_counts().sort_index().plot(style='.')

Total Fwd Packets

In [ ]:
data[' Total Fwd Packets'].value_counts()

In [ ]:
data[' Total Fwd Packets'].value_counts().sort_index().plot(style='.')

Total Backward Packets

In [ ]:
data[' Total Backward Packets'].value_counts()

In [ ]:
data[' Total Backward Packets'].value_counts().sort_index().plot(style='.')

Total Length of Fwd Packets

In [ ]:
data['Total Length of Fwd Packets'].value_counts()

In [ ]:
data['Total Length of Fwd Packets'].value_counts().sort_index().plot(style='.')

 Total Length of Bwd Packets

In [ ]:
data[' Total Length of Bwd Packets'].value_counts()

In [ ]:
data[' Total Length of Bwd Packets'].value_counts().sort_index().plot(style='.')

 Fwd Packet Length Max 	

In [ ]:
data[' Fwd Packet Length Max'].value_counts()

In [ ]:
data[' Fwd Packet Length Max'].value_counts().sort_index().plot(style='.')

 Fwd Packet Length Min

In [ ]:
data[' Fwd Packet Length Min'].value_counts()

In [ ]:
data[' Fwd Packet Length Min'].value_counts().sort_index().plot(style='.')

 Fwd Packet Length Mean

In [ ]:
data[' Fwd Packet Length Mean'].value_counts()

In [ ]:
data[' Fwd Packet Length Mean'].value_counts().sort_index().plot(style='.')

 Fwd Packet Length Std

In [ ]:
data[' Fwd Packet Length Std'].value_counts()

In [ ]:
data[' Fwd Packet Length Std'].value_counts().sort_index().plot(style='.')

Bwd Packet Length Max 

In [ ]:
data['Bwd Packet Length Max'].value_counts()

In [ ]:
data['Bwd Packet Length Max'].value_counts().sort_index().plot(style='.')

 Bwd Packet Length Min

In [ ]:
data[' Bwd Packet Length Min'].value_counts()

In [ ]:
data[' Bwd Packet Length Min'].value_counts().sort_index().plot(style='.')

 Bwd Packet Length Mean

In [ ]:
data[' Bwd Packet Length Mean'].value_counts()

In [ ]:
data[' Bwd Packet Length Mean'].value_counts().sort_index().plot(style='.')

 Bwd Packet Length Std

In [ ]:
data[' Bwd Packet Length Std'].value_counts()

In [ ]:
data[' Bwd Packet Length Std'].value_counts().sort_index().plot(style='.')

Flow Bytes/s 

In [ ]:
data['Flow Bytes/s'].value_counts()

In [ ]:
data['Flow Bytes/s'].value_counts().sort_index().plot(style='.')

 Flow Packets/s

In [ ]:
data[' Flow Packets/s'].value_counts()

In [ ]:
data[' Flow Packets/s'].value_counts().sort_index().plot(style='.')

 Flow IAT Mean

In [ ]:
data[' Flow IAT Mean'].value_counts()

In [ ]:
data[' Flow IAT Mean'].value_counts().sort_index().plot(style='.')

 Flow IAT Std

In [ ]:
data[' Flow IAT Std'].value_counts()

In [ ]:
data[' Flow IAT Std'].value_counts().sort_index().plot(style='.')

 Flow IAT Max

In [ ]:
data[' Flow IAT Max'].value_counts()

In [ ]:
data[' Flow IAT Max'].value_counts().sort_index().plot(style='.')

 Flow IAT Min

In [ ]:
data[' Flow IAT Min'].value_counts()

In [ ]:
data[' Flow IAT Min'].value_counts().sort_index().plot(style='.')

Fwd IAT Total

In [ ]:
data['Fwd IAT Total'].value_counts()

In [ ]:
data['Fwd IAT Total'].value_counts().sort_index().plot(style='.')

Fwd IAT Mean

In [ ]:
data[' Fwd IAT Mean'].value_counts()

In [ ]:
data[' Fwd IAT Mean'].value_counts().sort_index().plot(style='.')

Fwd IAT Std

In [ ]:
data[' Fwd IAT Std'].value_counts()

In [ ]:
data[' Fwd IAT Std'].value_counts().sort_index().plot(style='.')

Fwd IAT Max

In [ ]:
data[' Fwd IAT Max'].value_counts()

In [ ]:
data[' Fwd IAT Max'].value_counts().sort_index().plot(style='.')

Fwd IAT Min

In [ ]:
data[' Fwd IAT Min'].value_counts()

In [ ]:
data[' Fwd IAT Min'].value_counts().sort_index().plot(style='.')